# Building a Basic Product Recommender with TorchRec

In [ ]:
import torch
import torchrec
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple
from torchrec.sparse.jagged_tensor import KeyedJaggedTensor
from utils.data_generators import TorchRecDataGenerator
from utils.debugging import TorchRecDebugger
from utils.benchmark import TorchRecBenchmark

## Define Recommender Model

In [ ]:
class ProductRecommender(torch.nn.Module):
    """Two-tower model for product recommendations"""
    def __init__(
        self,
        num_products: int,
        num_categories: int,
        embedding_dim: int = 64,
        hidden_dim: int = 128,
    ):
        super().__init__()
        
        # Define embedding tables
        self.embedding_tables = torchrec.EmbeddingBagCollection(
            tables=[
                torchrec.EmbeddingBagConfig(
                    name="product_embeddings",
                    embedding_dim=embedding_dim,
                    num_embeddings=num_products,
                    feature_names=["product_id"],
                ),
                torchrec.EmbeddingBagConfig(
                    name="category_embeddings",
                    embedding_dim=embedding_dim,
                    num_embeddings=num_categories,
                    feature_names=["category_id"],
                ),
                torchrec.EmbeddingBagConfig(
                    name="product_history",
                    embedding_dim=embedding_dim,
                    num_embeddings=num_products,
                    feature_names=["history"],
                ),
            ],
            device=torch.device("meta"),  # Start on meta device
        )
        
        # User tower (processes user history)
        self.user_tower = torch.nn.Sequential(
            torch.nn.Linear(embedding_dim * 2, hidden_dim),  # Combine history + category
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, embedding_dim),
        )
        
        # Item tower (processes candidate items)
        self.item_tower = torch.nn.Sequential(
            torch.nn.Linear(embedding_dim * 2, hidden_dim),  # Combine product + category
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, embedding_dim),
        )
        
    def forward(
        self,
        user_features: KeyedJaggedTensor,
        item_features: KeyedJaggedTensor,
    ) -> torch.Tensor:
        # Get embeddings
        user_embeddings = self.embedding_tables(user_features)
        item_embeddings = self.embedding_tables(item_features)
        
        # Process user features
        user_history = user_embeddings.to_dict()["history"]
        user_categories = user_embeddings.to_dict()["category_id"]
        user_combined = torch.cat([user_history, user_categories], dim=1)
        user_vector = self.user_tower(user_combined)
        
        # Process item features
        item_product = item_embeddings.to_dict()["product_id"]
        item_category = item_embeddings.to_dict()["category_id"]
        item_combined = torch.cat([item_product, item_category], dim=1)
        item_vector = self.item_tower(item_combined)
        
        # Compute similarity scores
        return torch.matmul(user_vector, item_vector.t())

## Generate Synthetic Data

In [ ]:
class RecommenderDataGenerator:
    """Generate synthetic data for recommendation system"""
    def __init__(
        self,
        num_users: int,
        num_products: int,
        num_categories: int,
        max_history_length: int = 10,
    ):
        self.num_users = num_users
        self.num_products = num_products
        self.num_categories = num_categories
        self.max_history_length = max_history_length
        
        # Generate product categories
        self.product_categories = torch.randint(
            0, num_categories, (num_products,)
        )
    
    def generate_user_features(self, batch_size: int) -> KeyedJaggedTensor:
        """Generate user features including history"""
        # Generate random history lengths
        history_lengths = torch.randint(
            1, self.max_history_length + 1, (batch_size,)
        )
        
        # Generate product history
        history_values = torch.randint(
            0, self.num_products, (history_lengths.sum(),)
        )
        
        # Get categories for history items
        category_values = self.product_categories[history_values]
        
        return KeyedJaggedTensor.from_lengths_sync(
            keys=["history", "category_id"],
            values=torch.cat([history_values, category_values]),
            lengths=torch.cat([history_lengths, history_lengths])
        )
    
    def generate_item_features(self, batch_size: int) -> KeyedJaggedTensor:
        """Generate candidate item features"""
        # Sample random products
        products = torch.randint(0, self.num_products, (batch_size,))
        
        # Get their categories
        categories = self.product_categories[products]
        
        return KeyedJaggedTensor.from_lengths_sync(
            keys=["product_id", "category_id"],
            values=torch.cat([products, categories]),
            lengths=torch.ones(2 * batch_size)  # One value per feature
        )
    
    def generate_batch(
        self,
        batch_size: int,
        num_negatives: int = 4,
    ) -> Tuple[KeyedJaggedTensor, KeyedJaggedTensor, torch.Tensor]:
        """Generate complete training batch with negatives"""
        # Generate user features
        user_features = self.generate_user_features(batch_size)
        
        # Generate positive items
        pos_items = self.generate_item_features(batch_size)
        
        # Generate negative items
        neg_items = self.generate_item_features(batch_size * num_negatives)
        
        # Create labels (1 for positive, 0 for negative)
        labels = torch.cat([
            torch.ones(batch_size),
            torch.zeros(batch_size * num_negatives)
        ])
        
        return user_features, pos_items, neg_items, labels

## Training Loop

In [ ]:
class RecommenderTrainer:
    """Trainer for product recommender"""
    def __init__(
        self,
        model: ProductRecommender,
        learning_rate: float = 0.001,
        device: str = "cuda",
    ):
        self.model = model.to(device)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        self.device = device
        self.debugger = TorchRecDebugger()
        
    def train_step(
        self,
        user_features: KeyedJaggedTensor,
        pos_items: KeyedJaggedTensor,
        neg_items: KeyedJaggedTensor,
        labels: torch.Tensor,
    ) -> float:
        self.optimizer.zero_grad()
        
        # Move inputs to device
        user_features = user_features.to(self.device)
        pos_items = pos_items.to(self.device)
        neg_items = neg_items.to(self.device)
        labels = labels.to(self.device)
        
        # Combine positive and negative items
        all_items = KeyedJaggedTensor.concat([pos_items, neg_items])
        
        # Forward pass
        scores = self.model(user_features, all_items)
        
        # Compute loss
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            scores.view(-1), labels.float()
        )
        
        # Backward pass
        loss.backward()
        
        # Update weights
        self.optimizer.step()
        
        return loss.item()
    
    def train_epoch(
        self,
        data_generator: RecommenderDataGenerator,
        batch_size: int,
        num_batches: int,
    ) -> List[float]:
        losses = []
        
        for i in range(num_batches):
            # Generate batch
            user_features, pos_items, neg_items, labels = \
                data_generator.generate_batch(batch_size)
            
            # Train step
            loss = self.train_step(user_features, pos_items, neg_items, labels)
            losses.append(loss)
            
            if i % 10 == 0:
                print(f"Batch {i}, Loss: {loss:.4f}")
                
                # Monitor memory
                memory_stats = self.debugger.memory_status()
                print(f"Memory used: {memory_stats['allocated'] / 1e9:.2f}GB")
        
        return losses

## Evaluation

In [ ]:
class RecommenderEvaluator:
    """Evaluate recommender model"""
    def __init__(self, model: ProductRecommender, device: str = "cuda"):
        self.model = model
        self.device = device
    
    @torch.no_grad()
    def evaluate_batch(
        self,
        user_features: KeyedJaggedTensor,
        item_features: KeyedJaggedTensor,
        labels: torch.Tensor,
    ) -> Dict[str, float]:
        # Move to device
        user_features = user_features.to(self.device)
        item_features = item_features.to(self.device)
        labels = labels.to(self.device)
        
        # Get predictions
        scores = self.model(user_features, item_features)
        predictions = torch.sigmoid(scores)
        
        # Calculate metrics
        auc = self.compute_auc(predictions.view(-1), labels)
        precision = self.compute_precision_at_k(predictions.view(-1), labels, k=10)
        
        return {
            "auc": auc,
            "precision@10": precision,
        }
    
    def compute_auc(
        self,
        predictions: torch.Tensor,
        labels: torch.Tensor,
    ) -> float:
        # Sort predictions and corresponding labels
        sorted_pred, indices = torch.sort(predictions, descending=True)
        sorted_labels = labels[indices]
        
        # Compute AUC
        pos = sorted_labels.sum()
        neg = len(sorted_labels) - pos
        if pos == 0 or neg == 0:
            return 0.5
        
        pos_ranks = torch.where(sorted_labels == 1)[0]
        auc = (pos_ranks.float().sum() / pos - (pos + 1) / 2) / neg
        return auc.item()
    
    def compute_precision_at_k(
        self,
        predictions: torch.Tensor,
        labels: torch.Tensor,
        k: int,
    ) -> float:
        # Get top k predictions
        _, top_k = torch.topk(predictions, k)
        
        # Calculate precision
        return labels[top_k].float().mean().item()

## Complete Training Example

In [ ]:
def train_recommender(
    num_users: int = 10000,
    num_products: int = 1000,
    num_categories: int = 100,
    batch_size: int = 64,
    num_epochs: int = 5,
    device: str = "cuda",
):
    # Create data generator
    data_gen = RecommenderDataGenerator(
        num_users=num_users,
        num_products=num_products,
        num_categories=num_categories,
    )
    
    # Create model
    model = ProductRecommender(
        num_products=num_products,
        num_categories=num_categories,
    )
    
    # Create trainer and evaluator
    trainer = RecommenderTrainer(model, device=device)
    evaluator = RecommenderEvaluator(model, device=device)
    
    # Training loop
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}")
        
        # Train
        losses = trainer.train_epoch(
            data_generator=data_gen,
            batch_size=batch_size,
            num_batches=100,
        )
        
        # Evaluate
        user_features, pos_items, neg_items, labels = \
            data_gen.generate_batch(batch_size=1000)
        
        metrics = evaluator.evaluate_batch(
            user_features,
            KeyedJaggedTensor.concat([pos_items, neg_items]),
            labels,
        )
        
        print(f"Average Loss: {sum(losses) / len(losses):.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print(f"Precision@10: {metrics['precision@10']:.4f}")

## Model Analysis

In [ ]:
class ModelAnalyzer:
    """Analyze trained recommender model"""
    def __init__(self, model: ProductRecommender):
        self.model = model
    
    def analyze_embeddings(self):
        """Analyze embedding distributions"""
        stats = {}
        
        for name, param in self.model.named_parameters():
            if 'embedding' in name:
                stats[name] = {
                    'mean': param.mean().item(),
                    'std': param.std().item(),
                    'norm': param.norm().item(),
                }
        
        return stats
    
    def get_similar_products(
        self,
        product_id: int,
        top_k: int = 5,
        data_gen: RecommenderDataGenerator = None,
    ) -> List[Tuple[int, float]]:
        """Find similar products based on embeddings"""
        with torch.no_grad():
            # Get product embedding
            product_feature = KeyedJaggedTensor.from_lengths_sync(
                keys=["product_id"],
                values=torch.tensor([product_id]),
                lengths=torch.ones(1),
            )
            
            # Get category
            category = data_gen.product_categories[product_id].item()
            category_feature = KeyedJaggedTensor.from_lengths_sync(
                keys=["category_id"],
                values=torch.tensor([category]),
                lengths=torch.ones(1),
            )
            
            # Get combined features
            features = KeyedJaggedTensor.concat([product_feature, category_feature])
            
            # Get embedding
            embeddings = self.model.item_tower(
                torch.cat([
                    self.model.embedding_tables(features).to_dict()['product_id'],
                    self.model.embedding_tables(features).to_dict()['category_id'],
                ], dim=1)
            )
            
            # Compute similarities
            all_embeddings = []
            for pid in range(data_gen.num_products):
                pf = KeyedJaggedTensor.from_lengths_sync(
                    keys=["product_id"],
                    values=torch.tensor([pid]),
                    lengths=torch.ones(1),
                )
                cat = data_gen.product_categories[pid].item()
                cf = KeyedJaggedTensor.from_lengths_sync(
                    keys=["category_id"],
                    values=torch.tensor([cat]),
                    lengths=torch.ones(1),
                )
                f = KeyedJaggedTensor.concat([pf, cf])
                emb = self.model.item_tower(
                    torch.cat([
                        self.model.embedding_tables(f).to_dict()['product_id'],
                        self.model.embedding_tables(f).to_dict()['category_id'],
                    ], dim=1)
                )
                all_embeddings.append(emb)
            
            all_embeddings = torch.cat(all_embeddings, dim=0)
            similarities = torch.matmul(embeddings, all_embeddings.t())
            
            # Get top-k similar products
            top_values, top_indices = similarities[0].topk(top_k + 1)
            
            # Remove the query product itself
            mask = top_indices != product_id
            top_values = top_values[mask][:top_k]
            top_indices = top_indices[mask][:top_k]
            
            return list(zip(top_indices.tolist(), top_values.tolist()))

## Usage Example

In [ ]:
def main():
    # Initialize components
    print("Initializing recommender system...")
    
    num_products = 1000
    num_categories = 50
    batch_size = 64
    embedding_dim = 64
    
    # Create model and move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ProductRecommender(
        num_products=num_products,
        num_categories=num_categories,
        embedding_dim=embedding_dim
    ).to(device)
    
    # Create data generator
    data_gen = RecommenderDataGenerator(
        num_users=10000,
        num_products=num_products,
        num_categories=num_categories
    )
    
    # Train model
    print("\nTraining model...")
    trainer = RecommenderTrainer(model, device=device)
    evaluator = RecommenderEvaluator(model, device=device)
    
    for epoch in range(3):  # 3 epochs for demonstration
        print(f"\nEpoch {epoch + 1}")
        losses = trainer.train_epoch(data_gen, batch_size, num_batches=50)
        
        # Evaluation
        user_features, pos_items, neg_items, labels = data_gen.generate_batch(1000)
        metrics = evaluator.evaluate_batch(
            user_features,
            KeyedJaggedTensor.concat([pos_items, neg_items]),
            labels
        )
        
        print(f"Average Loss: {sum(losses) / len(losses):.4f}")
        print(f"Metrics: {metrics}")
    
    # Analyze model
    print("\nAnalyzing model...")
    analyzer = ModelAnalyzer(model)
    embedding_stats = analyzer.analyze_embeddings()
    print("\nEmbedding Statistics:")
    for name, stats in embedding_stats.items():
        print(f"{name}:")
        for stat_name, value in stats.items():
            print(f"  {stat_name}: {value:.4f}")
    
    # Generate recommendations
    print("\nGenerating sample recommendations...")
    sample_product_id = 42
    similar_products = analyzer.get_similar_products(
        sample_product_id,
        top_k=5,
        data_gen=data_gen
    )
    
    print(f"\nProducts similar to product {sample_product_id}:")
    for product_id, similarity in similar_products:
        category = data_gen.product_categories[product_id].item()
        print(f"Product {product_id} (Category {category}): {similarity:.4f}")

## Performance Optimization

In [ ]:
def optimize_performance():
    """Demonstrate performance optimization techniques"""
    debugger = TorchRecDebugger()
    benchmark = TorchRecBenchmark()
    
    print("\nPerformance optimization tips:")
    
    # 1. Memory optimization
    print("\n1. Memory Management:")
    memory_tips = {
        "Use meta device": "Initialize model on meta device first",
        "Batch size tuning": "Adjust batch size based on available memory",
        "Gradient checkpointing": "Use if memory is constrained",
        "Clear cache": "Regularly clear unused memory"
    }
    for tip, description in memory_tips.items():
        print(f"- {tip}: {description}")
    
    # 2. Computation optimization
    print("\n2. Computation Optimization:")
    compute_tips = {
        "Mixed precision": "Use torch.cuda.amp for faster training",
        "Parallel data loading": "Use multiple workers for data loading",
        "Embedding sharing": "Share embeddings when appropriate",
        "Batch compilation": "Use torch.compile for faster execution"
    }
    for tip, description in compute_tips.items():
        print(f"- {tip}: {description}")

## Production Checklist

In [ ]:
def production_checklist():
    """Checklist for production deployment"""
    checklist = {
        "Model Preparation": [
            "Quantize embeddings for inference",
            "Export model to TorchScript",
            "Implement model versioning",
            "Set up monitoring"
        ],
        "Data Pipeline": [
            "Implement efficient data loading",
            "Set up feature preprocessing",
            "Handle missing values",
            "Implement data validation"
        ],
        "Performance": [
            "Optimize batch size",
            "Implement caching",
            "Monitor memory usage",
            "Set up performance logging"
        ],
        "Monitoring": [
            "Track recommendation quality",
            "Monitor resource usage",
            "Set up alerting",
            "Implement A/B testing"
        ]
    }
    
    print("\nProduction Deployment Checklist:")
    for category, items in checklist.items():
        print(f"\n{category}:")
        for item in items:
            print(f"- {item}")

if __name__ == "__main__":
    main()
    optimize_performance()
    production_checklist()